# DayTime Checkout

This notebook is intended to run a series of tests to ensure the system responds as expected.
It begins with LATISS, but can also be used with the telescope ATCS components only.

In [ ]:
import asyncio
import time
import os
import numpy as np
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

Helper function

In [ ]:
def check_evt_time(evt):
    event_time = Time(evt.private_sndStamp, format="unix", scale="tai")
    event_time.format = "iso"
    event_time = event_time.utc
    return event_time

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

## Indicate if LATISS is part of the checkout

In [ ]:
with_latiss=True

## Enable LATISS

In [ ]:
if with_latiss:
    await latiss.enable()

## Check LATISS filters

In [ ]:
available_setup = await latiss.get_available_instrument_setup()
logger.info(f'\n The available filters are {available_setup[0]} \n and gratings are {available_setup[1]} ')

If ATCamera and/or ATOODS don't come up, see section `Quick Troubleshooting` for support commands. 

## LATISS image and ingestion verification

### Bias verification

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_bias(nbias=5)
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    ingest_event_time = check_evt_time(ingest_event)
    logger.info(f'The last ingested image was \n \t {ingest_event.obsid} at {ingest_event_time} UT')
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

In [ ]:
await latiss.take_bias(nbias=20)

In [ ]:
await latiss.take_darks(30.0, ndarks=20)

In [ ]:
await latiss.setup_instrument(filter='SDSSg_65mm', grating='empty_1')

In [ ]:
await latiss.take_flats(10.0, nflats=1)

In [ ]:
# Now take PTC sequence
for i in range(15,25):
    expTime = 2.0 * i
    await latiss.take_flats(expTime, nflats=2)
    

Check that the bias pops up in the monitor (https://roundtable.lsst.codes/rubintv/monitor_current)
If it fails, let Patrick or Merlin know

You can check that it ingested at USDF with this link: ******* <b>Add when available<b>


### Engineering test frame verification
This is analogous to a science frame

These are the filters and grating configurations available. 

In [ ]:
available_setup = await latiss.get_available_instrument_setup()
logger.info(f'\n The available filters are {available_setup[0]} \n and gratings are {available_setup[1]} ')

In [ ]:
await latiss.take_engtest(5, filter='empty_1',grating='holo4_003') 

In [ ]:
await latiss.take_engtest(5, filter='empty_1',grating='ronchi170lpmm')

In [ ]:
await latiss.take_engtest(5, filter='empty_1',grating='blue300lpmm_qn1')

In [ ]:
await latiss.take_engtest(5, filter='empty_1',grating='empty_1')

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_engtest(5, filter='empty_1',grating='empty_1')
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    ingest_event_time = check_evt_time(ingest_event)
    inst_setup = await latiss.get_setup()
    logger.info(f'The last ingested image was \n \t {ingest_event.obsid} at {ingest_event_time.utc} UT \n \t with {inst_setup[0]} filter and {inst_setup[1]} grating ')
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

## Enable ATCS

In [ ]:
await atcs.enable()

If any of the ATCS CSCs failed to transition to enabled state with the above command, see `Quick Troubleshooting` section for more commands. 

## Pneumatics Functionality Verification

Check ATPneumatics air pressure. 

In [ ]:
await atcs.open_valves()
pressure = await atcs.rem.atpneumatics.tel_mainAirSourcePressure.next(flush=True, timeout=5)
if pressure.pressure > 300000:
    logger.info(f'Air pressure is {pressure.pressure:0.0f} Pascals, which is fine.')
else:
    raise AssertionError(f'Air pressure is {pressure.pressure}, which is too low. It needs to be between ~275790 and ~413000 Pascals (40 and 60 PSI). \n Check that compressor and dryer are running. Then check that the regulator inside the pier is set correctly. ')

Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above).<br>
Note there is a race condition in the ATPneumatics that might result in this failing, so you'll have to run in twice

In [ ]:
cmd = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
cmd_time = check_evt_time(cmd)
logger.info(f'ATAOS corrections enabled -- {cmd.result} at {cmd_time} UT')

In [ ]:
m1_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
logger.info(f'M1 Air pressure with enabled ATAOS corrections is {m1_pressure.pressure:0.0f} Pascals')

Turn off ATAOS correction(s), without spectrograph (if the tests were skipped above)

In [ ]:
cmd = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
cmd_time = check_evt_time(cmd)
logger.info(f'Corrections disabled -- {cmd.result} at {cmd_time} UT')

Lower mirror back on hardpoints

In [ ]:
cmd = await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)
m1_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
logger.info(f'M1 air pressure when lowered back on hardpoints is {m1_pressure.pressure:0.0f} Pascals')

Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above). <br>
This should not ever fail.

In [ ]:
cmd = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
cmd_time = check_evt_time(cmd)
logger.info(f'ATAOS corrections enabled -- {cmd.result} at {cmd_time} UT')

## Start Telescope testing

Turn off dome following

The commands below will put the telescope in positionn for the new dome flat illuminator.

In [ ]:
await atcs.disable_dome_following()

In [ ]:
await atcs.slew_dome_to(355.0)

### Test Point AzEl

In [ ]:
start_az=10
start_el=75.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

In [ ]:
start_az=180.7
start_el=39.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

In [ ]:
# SDSSr_65mm - 10 sec, OG550_65mm_1 - 4 sec, BG40_65mm_1 - 6 sec.

await latiss.take_flats(10.0, 1, filter='SDSSg_65mm', grating='empty_1')

In [ ]:
start_az=180.7
start_el=39.0
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False, rot_type=RotType.PhysicalSky)
await latiss.take_engtest(5, filter='empty_1',grating='empty_1')

In [ ]:
# Stop tracking
await atcs.stop_tracking()

### Test Sidereal tracking
but do so starting from the same position

In [ ]:
coord=atcs.radec_from_azel(az=start_az+5, el=start_el+5)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False, rot_type=RotType.PhysicalSky)

In [ ]:
await latiss.take_engtest(5, filter='empty_1',grating='empty_1')

In [ ]:
# Stop tracking
await atcs.stop_tracking()

## Verify dome functions

Check that dome moves


In [ ]:
await atcs.slew_dome_to(13.0)

In [ ]:
await atcs.home_dome(force=True)

In [ ]:
dome_az = await atcs.rem.atdome.tel_position.next(flush=True,timeout=10)
logger.info(f'Dome currently thinks it is at an azimuth position of {dome_az.azimuthPosition} degrees.\n Note the dome may not be properly homed at this time')

In [ ]:
d_az=15
await atcs.rem.atdome.cmd_moveAzimuth.set_start(azimuth=dome_az.azimuthPosition+d_az)
dome_az = await atcs.rem.atdome.tel_position.next(flush=True,timeout=10)
logger.info(f'After the commanded {d_az} degrees motion, the dome is at an azimuth position of {dome_az.azimuthPosition} degrees')

## Prepare telescope and dome for flatfield

### Setup telescope to run the "prepare for flats" script so it maximizes the internal functionality of the script

In [ ]:
# Turn off ATAOS correction(s)
cmd = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
cmd_time = check_evt_time(cmd)
logger.info(f'Corrections disabled -- {cmd.result} at {cmd_time} UT')

In [ ]:
# Put mirror back on the hardpoints
cmd = await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)
m1_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
logger.info(f'M1 air pressure when lowered back on hardpoints is {m1_pressure.pressure:0.0f} Pascals')

In [ ]:
# Shut off the valves so we can test the pneumatics
cmd = await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()
cmd = await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()
cmd_time = check_evt_time(cmd)
logger.info(f'Valves closed -- {cmd.result} at {cmd_time} UT')

In [ ]:
# Open mirror covers and vents
# Note that there is currently a race conditions that might make this fail in the first attempt
await atcs.open_m1_cover()
await atcs.open_m1_vent()

In [ ]:
# Close mirror covers and vents
await atcs.close_m1_cover()
await atcs.close_m1_vent()

### Run prepare_for_flats
First we home the dome to make sure it knows where it is.

In [ ]:
await atcs.home_dome()
dome_az = await atcs.rem.atdome.tel_position.next(flush=True,timeout=10)
logger.info(f'The dome is homed at an azimuth position of {dome_az.azimuthPosition} degrees.')

In [ ]:
await atcs.prepare_for_flatfield()

## Perform slew of Full Observatory
This will also take test images

Now close the mirror cover just to keep things safe - this will move the telescope to a higher position (El~70)

In [ ]:
await atcs.close_m1_cover()
await atcs.close_m1_vent()

Turn on dome following which will align the dome with the telescope

In [ ]:
await atcs.enable_dome_following()

Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above). <br>
Note there is a race condition in the ATPneumatics that might result in this failing. <br>
If it fails, you'll have to run in twice

In [ ]:
cmd = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
cmd_time = check_evt_time(cmd)
logger.info(f'ATAOS corrections enabled -- {cmd.result} at {cmd_time} UT')

Now start tracking a siderial target, but starting from the same position. <br>
Then take a test image engineering test frame (analogous to a science frame) and make sure it lands

In [ ]:
start_az=45
start_el=45
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False, rot_type=RotType.PhysicalSky)

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter=3, grating=3)
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    ingest_event_time = check_evt_time(ingest_event)
    inst_setup = await latiss.get_setup()
    logger.info(f'The last ingested image was \n \t {ingest_event.obsid} at {ingest_event_time} UT \n \t with {inst_setup[0]} filter and {inst_setup[1]} grating ')
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

Pause here and let it track for at least a couple minutes

### Slew to target #2
Then take an image with a different spectrograph setup.

In [ ]:
start_az=0.0
start_el=80
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False, rot_type=RotType.PhysicalSky)

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter=0, grating=0)
    iingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    ingest_event_time = check_evt_time(ingest_event)
    inst_setup = await latiss.get_setup()
    logger.info(f'The last ingested image was \n \t {ingest_event.obsid} at {ingest_event_time} UT \n \t with {inst_setup[0]} filter and {inst_setup[1]} grating ')
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

## M3 functionality 

M3 verification is not required every time this daytime checkout is run. Check the DayTime Checkout historical data to corroborate if this section is meant to run today.  

Confirm M3 mirror rotation is operational and its valve is routinely exercized by running the following commands. 

First, move to the zenith where the Nasmyth motor can function with ease.  

In [ ]:
start_az=.0
start_el=80.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

Once the enumerations have been updated in the class M3ExitPort https://github.com/lsst-ts/ts_idl/blob/2add7a88bba16f2b13448c3ea6f78ef6d988ce83/python/lsst/ts/idl/enums/ATMCS.py#L34-L37, substitute the following five cells with: 

`from lsst.ts.idl.enums.ATMCS import M3ExitPort`
Move to the Nasmyth1 port
`await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort.NASMYTH1)`
and back to Nasmyth 2
`await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort.NASMYTH2)`

In [ ]:
from lsst.ts.idl.enums.ATMCS import M3ExitPort

In [ ]:
await atcs.rem.atmcs.evt_m3PortSelected.aget()

In [ ]:
M3ExitPort = {'NASMYTH1': 1, 'NASMYTH2':2}

Move to the Nasmyth1 port

In [ ]:
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort['NASMYTH1'])
m3 = await atcs.rem.atmcs.evt_m3PortSelected.aget()
m3_time = check_evt_time(m3)
logger.info(f'M3 Port selected -- {m3.selected} at {m3_time} UT')

and back to Nasmyth 2

In [ ]:
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort['NASMYTH2'])
m3 = await atcs.rem.atmcs.evt_m3PortSelected.aget()
m3_time = check_evt_time(m3)
logger.info(f'M3 Port selected -- {m3.selected} at {m3_time} UT')

## Shutdown all ATCS components

In [ ]:
await atcs.shutdown()

In [ ]:
await atcs.standby()

## Put LATISS in standby

In [ ]:
await latiss.standby()

# Quick Troubleshooting

## LATISS CSCs failed to transition to ENABLED state. 
The current DDS configuration results in history being lost when CSCs are restarted and/or a small network outage occurs. If this happens, then to receive the summaryState event, which is required by salobj.get_summary_state (and used extensively in the atcs), it is necessary to manually change the state of a CSC. The commands below can be used to perform this action.

A new DDS configuration is now ready to be tested and will hopefully be released in the near future.

### ATOODs didn't come up

In [ ]:
tmp=await latiss.rem.atoods.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atoods, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(latiss.rem.atoods, salobj.State.ENABLED)

### ATSpectrograph didn't come up

In [ ]:
tmp=await latiss.rem.atspectrograph.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.ENABLED)

### ATHeaderService didn't come up

In [ ]:
tmp=await latiss.rem.atheaderservice.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atheaderservice, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(latiss.rem.atoods, salobj.State.ENABLED)

### ATCamera didn't come up

In [ ]:
tmp=await latiss.rem.atcamera.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

## ATCS CSCs failed to transition to ENABLED state. 

### ATHexapod 

In [ ]:
tmp=await atcs.rem.athexapod.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY)

### ATPneumatics

In [ ]:
tmp=await atcs.rem.atpneumatics.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)

### ATPtg 

In [ ]:
tmp=await atcs.rem.atptg.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

### ATMCS 

In [ ]:
tmp=await atcs.rem.atmcs.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

### Nov 3, 2022 - taking calibration images with new configuration.

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
await atcs.rem.atdome.cmd_openShutter.set_start()

In [ ]:
await atcs.rem.atdome.cmd_closeShutter.set_start()

In [ ]:
await atcs.rem.ataos.cmd_offset.set_start(z=+0.5)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)

In [ ]:
    tmp = 
print(tmp)

In [ ]:
await atcs.rem.atptg.cmd_offsetClear.start()

In [ ]:
#await atcs.rem.atdome.cmd_openShutter.set_start()

In [ ]:
await atcs.open_dome_shutter()

In [ ]:
await atcs.close_dome()

In [ ]:
atcs.rem.atdome.cmd_moveAzimuth?